# Tratamento da Base de Dados

Importação das bibliotecas

In [1]:
import pandas as pd

## Padronização da Base

Carregamento da base de dados

In [2]:
df = pd.read_csv("./dataset.csv")

In [3]:
df.drop('loan_id', axis=1, inplace=True)
df.head()

,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,loan_status
0,2,Graduate,No,9600000,29900000,12,778,2400000,17600000,22700000,8000000,Approved
1,0,Not Graduate,Yes,4100000,12200000,8,417,2700000,2200000,8800000,3300000,Rejected
2,3,Graduate,No,9100000,29700000,20,506,7100000,4500000,33300000,12800000,Rejected
3,3,Graduate,No,8200000,30700000,8,467,18200000,3300000,23300000,7900000,Rejected
4,5,Not Graduate,Yes,9800000,24200000,20,382,12400000,8200000,29400000,5000000,Rejected


Acima, é realizada a exclusão da coluna que armazena o identificador único de cada empréstimo

In [4]:
df.columns = ['number_of_dependents', 'is_graduated', 'is_self_employed', 'annual_income', 'loan_amount', \
              'loan_term', 'credit_score', 'residential_assets', 'commercial_assets', 'luxury_assets', \
              'bank_assets', 'loan_status']

In [5]:
df.head()

,number_of_dependents,is_graduated,is_self_employed,annual_income,loan_amount,loan_term,credit_score,residential_assets,commercial_assets,luxury_assets,bank_assets,loan_status
0,2,Graduate,No,9600000,29900000,12,778,2400000,17600000,22700000,8000000,Approved
1,0,Not Graduate,Yes,4100000,12200000,8,417,2700000,2200000,8800000,3300000,Rejected
2,3,Graduate,No,9100000,29700000,20,506,7100000,4500000,33300000,12800000,Rejected
3,3,Graduate,No,8200000,30700000,8,467,18200000,3300000,23300000,7900000,Rejected
4,5,Not Graduate,Yes,9800000,24200000,20,382,12400000,8200000,29400000,5000000,Rejected


É realizada a renomeação das colunas para nomes mais descritivos

In [6]:
df['is_graduated'] = df['is_graduated'].replace({' Graduate': 'yes', ' Not Graduate': 'no'})

In [7]:
df['is_self_employed'] = df['is_self_employed'].str.lower().str.lstrip()
df['loan_status'] = df['loan_status'].str.lower().str.lstrip()

In [8]:
df.head()

,number_of_dependents,is_graduated,is_self_employed,annual_income,loan_amount,loan_term,credit_score,residential_assets,commercial_assets,luxury_assets,bank_assets,loan_status
0,2,yes,no,9600000,29900000,12,778,2400000,17600000,22700000,8000000,approved
1,0,no,yes,4100000,12200000,8,417,2700000,2200000,8800000,3300000,rejected
2,3,yes,no,9100000,29700000,20,506,7100000,4500000,33300000,12800000,rejected
3,3,yes,no,8200000,30700000,8,467,18200000,3300000,23300000,7900000,rejected
4,5,no,yes,9800000,24200000,20,382,12400000,8200000,29400000,5000000,rejected


Acima, os valores da coluna is_graduated foram padronizados para 'yes' e 'no' e os valores das outras colunas categóricas foram transformados para terem apenas letras minúsculas e o espaço em branco na frente da palavra foi removido

In [9]:
variaveis_monetarias = ['annual_income', 'loan_amount', 'residential_assets', 'commercial_assets', 'luxury_assets', 'bank_assets']

In [10]:
df[variaveis_monetarias] = df[variaveis_monetarias].astype(float)

In [11]:
df[variaveis_monetarias] = df[variaveis_monetarias] / 100

In [12]:
df.head()

,number_of_dependents,is_graduated,is_self_employed,annual_income,loan_amount,loan_term,credit_score,residential_assets,commercial_assets,luxury_assets,bank_assets,loan_status
0,2,yes,no,96000.0,299000.0,12,778,24000.0,176000.0,227000.0,80000.0,approved
1,0,no,yes,41000.0,122000.0,8,417,27000.0,22000.0,88000.0,33000.0,rejected
2,3,yes,no,91000.0,297000.0,20,506,71000.0,45000.0,333000.0,128000.0,rejected
3,3,yes,no,82000.0,307000.0,8,467,182000.0,33000.0,233000.0,79000.0,rejected
4,5,no,yes,98000.0,242000.0,20,382,124000.0,82000.0,294000.0,50000.0,rejected


In [13]:
print(df[variaveis_monetarias].dtypes)

annual_income         float64
loan_amount           float64
residential_assets    float64
commercial_assets     float64
luxury_assets         float64
bank_assets           float64
dtype: object


As variáveis que armazenam valores monetários tiveram seus tipos convertidos para float e os valores foram divididos por 100 para reduzir a ordem de grandeza

## Remoção de Outliers

In [14]:
variaveis_numericas = ['number_of_dependents', 'annual_income', 'loan_amount', 'loan_term', 'credit_score', \
                       'residential_assets', 'commercial_assets', 'luxury_assets', 'bank_assets']

In [15]:
while True:
    indices_outliers = set()

    for coluna in variaveis_numericas:
        q1 = df[coluna].quantile(0.25)
        q3 = df[coluna].quantile(0.75)
        iqr = q3 - q1

        limite_inferior = q1 - 1.5 * iqr
        limite_superior = q3 + 1.5 * iqr

        outliers = df[(df[coluna] < limite_inferior) | (df[coluna] > limite_superior)].index
        indices_outliers.update(outliers)

    if not indices_outliers:
        break

    df.drop(index=indices_outliers, inplace=True)

In [16]:
df.shape

(4108, 12)

Acima, a remoção de outliers foi realizada de forma iterativa, até que nenhum outlier fosse detectado com base no cálculo do IQR. Ao fim do processo, restaram 4108 registros na base de dados

In [17]:
df.head()

,number_of_dependents,is_graduated,is_self_employed,annual_income,loan_amount,loan_term,credit_score,residential_assets,commercial_assets,luxury_assets,bank_assets,loan_status
1,0,no,yes,41000.0,122000.0,8,417,27000.0,22000.0,88000.0,33000.0,rejected
2,3,yes,no,91000.0,297000.0,20,506,71000.0,45000.0,333000.0,128000.0,rejected
3,3,yes,no,82000.0,307000.0,8,467,182000.0,33000.0,233000.0,79000.0,rejected
4,5,no,yes,98000.0,242000.0,20,382,124000.0,82000.0,294000.0,50000.0,rejected
5,0,yes,yes,48000.0,135000.0,10,319,68000.0,83000.0,137000.0,51000.0,rejected


In [18]:
df.to_csv('dataset_tratado.csv', index=False)

O dataframe tratado foi salvo para ser utilizado nas próximas etapas do projeto